In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

In [ ]:
print(transformers.__version__)
print(tf.__version__)

4.44.2
2.17.0


In [ ]:
#pandas read_excel 함수 사용 xlrd, openpyxl 설치
!pip install openpyxl

In [ ]:
comment_train = pd.read_excel('https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_train.xlsx', engine='openpyxl')
comment_test = pd.read_excel('https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_test.xlsx', engine='openpyxl')

In [ ]:
comment_train.head()

,Unnamed: 0,data,label
0,0,재미는 있는데 시간이 짧은게 아쉽네요~,긍정
1,1,"OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄",부정
2,2,스토리가 너무 딱딱해서 별로였음,부정
3,3,프로그램A 화이팅하세요!!,긍정
4,4,높은 곳에 올라가는 모습이 너무 위험해 보여요.,부정


In [ ]:
comment_test.count()

,0
Unnamed: 0,101
data,101
label,101


In [ ]:
#comment = pd.concat([comment_train, comment_test])
comment = comment_train

In [ ]:
comment.head()

,Unnamed: 0,data,label
0,0,재미는 있는데 시간이 짧은게 아쉽네요~,긍정
1,1,"OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄",부정
2,2,스토리가 너무 딱딱해서 별로였음,부정
3,3,프로그램A 화이팅하세요!!,긍정
4,4,높은 곳에 올라가는 모습이 너무 위험해 보여요.,부정


In [ ]:
comment.isnull().sum()

,0
Unnamed: 0,0
data,0
label,0


In [ ]:
# label 변환
#comment['label'] = comment['label'].replace(['부정', '긍정'],[0,1])
comment['label'] = comment['label'].apply(lambda x: 0 if x == '부정' else 1)

In [ ]:
comment.tail()

,Unnamed: 0,data,label
246,246,영상F서비스로 간편하게 설치!좋아요\n우리 회사화이팅!,1
247,247,모든 업무에서 맡은바 업무에 서 최선을 다하는 모습이 좋습니다! 화이팅 입니다.,1
248,248,"사내방송 특성상 최근 이슈화 되거나, 언급이 자주되는 '키워드'를 중심으로 뉴스를 ...",0
249,249,방송 시간이 너무 길어요.,0
250,250,"처음 들어보는 말들이 많은데,, 설명이 없어서 힘드네요.",0


In [ ]:
comment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  251 non-null    int64 
 1   data        251 non-null    object
 2   label       251 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.0+ KB


In [ ]:
X = comment.data.to_list()
y = comment.label.to_list()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
len(X_train), len(X_test), len(y_train), len(y_test)

(200, 51, 200, 51)

In [ ]:
X_train[:2]

['우리 회사 기업 이미지 활동에도 세심한 할동과 노력이 필요할때 같습니다',
 '간략한 브리핑이나 아님 끝나고 나서 오늘 전달한 콘텐츠에 대한 정리를 한번 해주시면 한 눈에 딱 들어와서 좋을 듯 합니다.']

In [ ]:
bert_model = 'klue/bert-base'

In [ ]:
from transformers import AutoConfig, BertTokenizerFast, TFBertForSequenceClassification
tokenizer = BertTokenizerFast.from_pretrained(bert_model)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenizer.vocab_size

32000

In [ ]:
#tokenizer.vocab

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
print(train_encodings['input_ids'][0])

[2, 3616, 3769, 3646, 4661, 3746, 6509, 12541, 2470, 1892, 2328, 2145, 3973, 2052, 3677, 2085, 2775, 555, 2219, 3606, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(test_encodings['input_ids'][0])

[2, 4365, 2116, 16984, 2897, 4347, 2052, 2379, 3760, 22939, 2052, 2203, 2182, 18, 4243, 2116, 3760, 1521, 13486, 2182, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train))
train_dataset = train_dataset.shuffle(1000).batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test))
test_dataset = test_dataset.batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
config = AutoConfig.from_pretrained(bert_model)
config

BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

In [ ]:
#몇개로 분류할거냐 num_label = 2, 파이토치 학슨된 정보를 tensorflow 로 불러 from_pt=True
model = TFBertForSequenceClassification.from_pretrained(bert_model, num_labels=2, from_pt=True)
from transformers import TFAutoModelForSequenceClassification

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(train_dataset, epochs=10, batch_size=16, validation_data=(test_dataset))

Epoch 1/10
13/13 [==============================] - 58s 591ms/step - loss: 0.5465 - accuracy: 0.8150 - val_loss: 0.4230 - val_accuracy: 0.9216
Epoch 2/10
13/13 [==============================] - 5s 371ms/step - loss: 0.2964 - accuracy: 0.9750 - val_loss: 0.2611 - val_accuracy: 0.9412
Epoch 3/10
13/13 [==============================] - 5s 357ms/step - loss: 0.1464 - accuracy: 0.9850 - val_loss: 0.1624 - val_accuracy: 0.9608
Epoch 4/10
13/13 [==============================] - 5s 357ms/step - loss: 0.0671 - accuracy: 1.0000 - val_loss: 0.1257 - val_accuracy: 0.9608
Epoch 5/10
13/13 [==============================] - 5s 384ms/step - loss: 0.0351 - accuracy: 1.0000 - val_loss: 0.1135 - val_accuracy: 0.9608
Epoch 6/10
13/13 [==============================] - 5s 358ms/step - loss: 0.0217 - accuracy: 1.0000 - val_loss: 0.1038 - val_accuracy: 0.9608
Epoch 7/10
13/13 [==============================] - 5s 375ms/step - loss: 0.0155 - accuracy: 1.0000 - val_loss: 0.1011 - val_accuracy: 0.9804
Epoch

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 110618882 (421.98 MB)
Trainable params: 110618882 (421.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
comment_valid = comment_test.drop('label', axis=1)
comment_valid

,Unnamed: 0,data
0,0,유익한 정보 감사합니다. 취미로 바다에 자주 나가는 편인데 이런 기술이 있는지 몰랐네요.
1,1,발빠른 영입 수고 했네요 고객들이 좋아합니다.
2,2,프로그램C 주제가 우리의 기술력이 바탕이 된다는점~~..항상 응원합니다
3,3,언제나 밝은 미소로 열심히 근무하시는 청원 경찰 분들께 감사드립니다. 항상 건강하시...
4,4,안전사고에 대한 개념도 함께 소개 되었으면 더 완벽했을 것 같네요
...,...,...
96,96,작년에 프로그램A를 재밋게 봤던 시청자로서 올해의 미니드라마도 매우 기대가 됩니다....
97,97,프로그램C 잘 보았습니다. 모든일의 바탕은 안전인것 같습니다. 모두를 보호하는 최고...
98,98,위험한 시설에 대한 설명도 부탁드립니다
99,99,구체적으로 어떤 활동을 해왔었고 앞으로 어떤활동을 할건지 잘 설명해줬으면 좋았을 것...


In [ ]:
comment_valid['data']

,data
0,유익한 정보 감사합니다. 취미로 바다에 자주 나가는 편인데 이런 기술이 있는지 몰랐네요.
1,발빠른 영입 수고 했네요 고객들이 좋아합니다.
2,프로그램C 주제가 우리의 기술력이 바탕이 된다는점~~..항상 응원합니다
3,언제나 밝은 미소로 열심히 근무하시는 청원 경찰 분들께 감사드립니다. 항상 건강하시...
4,안전사고에 대한 개념도 함께 소개 되었으면 더 완벽했을 것 같네요
...,...
96,작년에 프로그램A를 재밋게 봤던 시청자로서 올해의 미니드라마도 매우 기대가 됩니다....
97,프로그램C 잘 보았습니다. 모든일의 바탕은 안전인것 같습니다. 모두를 보호하는 최고...
98,위험한 시설에 대한 설명도 부탁드립니다
99,구체적으로 어떤 활동을 해왔었고 앞으로 어떤활동을 할건지 잘 설명해줬으면 좋았을 것...


In [ ]:
# comment_valid의 데이터를 리스트로 변환
valid_texts = comment_valid['data'].to_list()

# 토크나이징
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True)

# valid_encodings를 TensorFlow 데이터셋으로 변환
valid_dataset = tf.data.Dataset.from_tensor_slices((dict(valid_encodings)))
valid_dataset = valid_dataset.batch(16)

# 예측 결과 얻기
predictions = model.predict(valid_dataset)

7/7 [==============================] - 3s 74ms/step


In [ ]:
predictions.logits

array([[-2.0256371 ,  2.8980608 ],
       [-2.4638686 ,  3.0682173 ],
       [-2.3047283 ,  3.1526093 ],
       [-2.421017  ,  2.9058363 ],
       [ 2.4947276 , -2.135936  ],
       [-2.39279   ,  3.0074892 ],
       [-2.3139982 ,  2.8844903 ],
       [ 2.7493455 , -2.296551  ],
       [ 2.530861  , -2.1216004 ],
       [ 2.564917  , -2.526381  ],
       [ 2.5758035 , -2.2003055 ],
       [-1.0125813 ,  2.1939633 ],
       [ 0.74981135, -0.10980676],
       [-2.3244004 ,  3.0861692 ],
       [-2.3703735 ,  3.2096646 ],
       [ 2.4523177 , -2.129935  ],
       [-2.390077  ,  3.098765  ],
       [ 1.4624766 , -0.89176434],
       [ 2.415533  , -2.1179934 ],
       [-2.3799906 ,  3.044531  ],
       [-2.3766313 ,  2.9987235 ],
       [-2.1951108 ,  2.9488156 ],
       [-2.4254668 ,  2.980787  ],
       [-2.4844823 ,  3.1769319 ],
       [ 2.9107084 , -2.4567668 ],
       [ 2.3056142 , -2.1497815 ],
       [-1.9648348 ,  2.8956995 ],
       [ 2.6260757 , -2.361945  ],
       [-2.4863863 ,

In [ ]:

# logits에서 클래스 예측값으로 변환
predicted_labels = np.argmax(predictions.logits, axis=1)

# 실제 레이블 변환 (부정=0, 긍정=1)
true_labels = comment_test['label'].apply(lambda x: 0 if x == '부정' else 1).values

# 정확도 계산
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(true_labels, predicted_labels)
print(f'정확도: {accuracy:.4f}')

# 상세한 분류 보고서 출력
print('\n분류 보고서:')
print(classification_report(true_labels, predicted_labels,
                          target_names=['부정', '긍정']))

# 예측 결과를 DataFrame으로 저장 (선택사항)
results_df = pd.DataFrame({
    '텍스트': comment_test['data'],
    '실제 레이블': comment_test['label'],
    '예측 레이블': ['부정' if label == 0 else '긍정' for label in predicted_labels]
})

정확도: 0.9604

분류 보고서:
              precision    recall  f1-score   support

          부정       0.97      0.93      0.95        42
          긍정       0.95      0.98      0.97        59

    accuracy                           0.96       101
   macro avg       0.96      0.96      0.96       101
weighted avg       0.96      0.96      0.96       101


예측 결과 샘플:
                                                 텍스트 실제 레이블 예측 레이블
0  유익한 정보 감사합니다. 취미로 바다에 자주 나가는 편인데 이런 기술이 있는지 몰랐네요.     긍정     긍정
1                          발빠른 영입 수고 했네요 고객들이 좋아합니다.     긍정     긍정
2            프로그램C 주제가 우리의 기술력이 바탕이 된다는점~~..항상 응원합니다     긍정     긍정
3  언제나 밝은 미소로 열심히 근무하시는 청원 경찰 분들께 감사드립니다. 항상 건강하시...     긍정     긍정
4               안전사고에 대한 개념도 함께 소개 되었으면 더 완벽했을 것 같네요     부정     부정


In [ ]:
print(results_df.head())

                                                 텍스트 실제 레이블 예측 레이블
0  유익한 정보 감사합니다. 취미로 바다에 자주 나가는 편인데 이런 기술이 있는지 몰랐네요.     긍정     긍정
1                          발빠른 영입 수고 했네요 고객들이 좋아합니다.     긍정     긍정
2            프로그램C 주제가 우리의 기술력이 바탕이 된다는점~~..항상 응원합니다     긍정     긍정
3  언제나 밝은 미소로 열심히 근무하시는 청원 경찰 분들께 감사드립니다. 항상 건강하시...     긍정     긍정
4               안전사고에 대한 개념도 함께 소개 되었으면 더 완벽했을 것 같네요     부정     부정


In [ ]:
#모델저장
#model.save('model.h5')
#results_df.to_csv('results.csv')

In [ ]:
# 예측 결과를 DataFrame으로 저장
results_df = pd.DataFrame({
    '텍스트': comment_test['data'],
    '실제 레이블': comment_test['label'],
    '예측 레이블': ['부정' if label == 0 else '긍정' for label in predicted_labels]
})

# 실제 레이블과 예측 레이블이 다른 경우만 필터링
incorrect_predictions = results_df[results_df['실제 레이블'] != results_df['예측 레이블']]

incorrect_predictions

,텍스트,실제 레이블,예측 레이블
32,요즘같이 우울한 시기에는 희망찬 주제로 부탁해요,부정,긍정
53,다양한 주제로 기획해주세요,부정,긍정
58,주위에 항상 있으나 있는듯 없는듯...중요한지도 고마운지도 모르고 지나치는 산소같은...,긍정,부정
71,인터뷰가 대본을 눈으로 보고 읽고있는 것 같아요,부정,긍정
